In [40]:
import argparse
from matplotlib import pyplot as plt
import numpy as np
import torch
import sys, os

from torch_geometric.data import Data as Data_g
from torch_geometric.data import DataLoader as DataLoader_G
from torch.utils.data import DataLoader, TensorDataset


# import path
sys.path.append("../")
from filepath import ABSOLUTE_PATH

sys.path.append(ABSOLUTE_PATH)
from src.train.heatpipe_meshgraph import load_data
from src.train.heatpipe import renormalize
from src.train.train import Trainer
from src.model.meshgraphnet import EncoderProcesserDecoder as MeshGraphNet
from src.utils.utils import (
    create_res,
    set_seed,
    get_time,
    save_config_from_args,
    get_parameter_net,
    find_max_min,
    relative_error,
)
import time

In [42]:
device = "cuda"
num_layers, hidden_channels = 4, 2412
message_passing_num = 6
node_input_size = 4
edge_input_size = 1
hidden_size = 256
output_size = 2412
model = MeshGraphNet(
    message_passing_num,
    node_input_size,
    edge_input_size,
    hidden_size=hidden_size,
    output_size=output_size,
).to(device)
model.load_state_dict(torch.load("../../results/heatpipe/surrogate/meshgraph_6_256/model-50.pt")["model"])

C:\Users\taozhan\AppData\Local\Temp\ipykernel_18924\4057717591.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("../../results/heatpipe/

<All keys matched successfully>

In [43]:
_, test_loader = load_data(ABSOLUTE_PATH, 200, 800, device)

In [44]:
for batch in test_loader:
    y_p = renormalize(model(batch).reshape(-1, 804, 3))
y_t = renormalize(batch.y.reshape(-1, 804, 3))  # .reshape(-1, 804, 3)

In [45]:
relative_error(y_t[..., 0], y_p[..., 0]), relative_error(y_t[..., 1:], y_p[..., 1:])

(0.0005833123197507619, 0.004266419302878069)

In [ ]:
from torch_geometric.data import Data

x_val = torch.tensor(np.load("../../data/heatpipe/x_val_element_as_one_node.npy")).to(device).float()



y_val_t = torch.tensor(np.load("../../data/heatpipe/y_val_element_as_one_node.npy")).to(device)



batch_batch = torch.zeros(size=(64,), device=device)
adj = torch.tensor(np.load("../../data/heatpipe/adj_val_element_as_one_node.npy").transpose(1, 0)).long().to(device)
n_edge = adj.shape[-1]
edge_attr = torch.zeros((n_edge, 1), device=device)
data = Data(x=x_val, y=y_val_t, edge_index=adj, edge_attr=edge_attr)
x_val.shape, adj.shape

(torch.Size([64, 4]), torch.Size([2, 168]))

In [47]:
y_val_p = renormalize(model(data).reshape(-1, 804, 3)).reshape(-1, 804, 3)
y_val_p.shape

torch.Size([64, 804, 3])

In [48]:
relative_error(y_val_t[..., 0], y_val_p[..., 0]), relative_error(y_val_t[..., 1:], y_val_p[..., 1:])

(0.012795742191389682, 0.023953305791562077)

In [21]:
test_loss = np.load("../../results/heatpipe/surrogate/GIN_4_1206/record.npy")
test_loss

array([[1.00000000e+00, 5.22917579e-03, 1.38625251e-02],
       [2.00000000e+00, 2.59271089e-03, 8.33722297e-03],
       [3.00000000e+00, 1.42485782e-03, 6.18609274e-03],
       [4.00000000e+00, 6.56058139e-04, 6.04053307e-03],
       [5.00000000e+00, 3.36065757e-04, 6.28599059e-03],
       [6.00000000e+00, 2.79266431e-04, 6.48554647e-03],
       [7.00000000e+00, 2.58250395e-04, 6.57488825e-03],
       [8.00000000e+00, 2.34310282e-04, 6.59090560e-03],
       [9.00000000e+00, 1.75298497e-04, 6.57350942e-03],
       [1.00000000e+01, 5.55696643e-05, 6.54480560e-03]])